In [1]:
with open("words", 'r') as file:
        lines_array = [line.strip() for line in file.readlines()]
# print(lines_array)
# print(len(lines_array))

In [2]:
from ollama import Client
from dotenv import load_dotenv
import random

def get_prompt():
    word1 = lines_array[random.randint(0, len(lines_array))]
    word2 = lines_array[random.randint(0, len(lines_array))]

    return f"Write a single sentence with between 8 and 10 words. Only output the sentence generated without preamble. Use the words {word1} and {word2} as a theme. Remember, the sentence must be no more than 10 words long."

def generate_response(client, prompt, model):
    seed = random.random()*1000
    system_prompt = "You are attempting to make a memorable phrase to be used as a key like a password. It must use most letters of the alphabet. No emojis." #f"Use the random seed {seed}" # SYSTEM_PROMPT + "\n".join(item[0] for item in most_similar_chunks)
    # print(seed)
    # print(prompt)
    options = {
        'seed': seed, 
        'temperature': seed/100 
    }
    response = client.chat(
        model,
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        options=options,
        stream = False
    )
    return response['message']['content']


# print(generate_response(client, get_prompt(), "gemma3:12b"))


In [3]:
class Sentence:
    def __init__(self, text):
        """
        Args:
            text (str): The text of the sentence.  Must be a string.
        
        Raises:
            TypeError: if input is not a string
        """
        if not isinstance(text, str):
            raise TypeError("Input must be a string.")

        if len(text) > 80:
            raise TypeError("No paragraphs.")
        
        self._text = text  # Use a protected attribute for encapsulation
        self._words = len(text.split(" "))
        self._missing = self.find_missing_letters()
        self._duplicates = self.identify_duplicate_letters()
        self._duplength = len(self._duplicates)
        self._dupkeys = "".join(self._duplicates.keys())

    def get_text(self):
        """
        Returns the text of the sentence.

        Returns:
            str: The sentence text.
        """
        return self._text

    def get_missing(self):
        return self._missing
    
    def get_missing_count(self):
        return len(self.get_missing())
      
    def get_duplicates(self):
        return self._duplicates
    
    def get_duplength(self):
        return self._duplength
    
    def get_dupkeys(self):
        return self._dupkeys
    
    def find_missing_letters(self):
        """
        Checks a string for the presence of all lowercase letters and identifies missing ones.

        Args:
            input_string: The string to test.

        Returns:
            A string containing the missing letters, or an empty string if all letters are present.
            Returns None if the input is not a string.
        """

        if not isinstance(self._text, str):
            return None

        alphabet = "abcdefghijklmnopqrstuvwxyz"
        missing_letters = ""
        
        # Convert the input string to lowercase for case-insensitive comparison
        input_string = self._text.lower()

        for letter in alphabet:
            if letter not in input_string:
                missing_letters += letter

        return missing_letters

    def identify_duplicate_letters(self):
        """
        Identifies duplicate letters in a string and counts their occurrences.

        Args:
            input_string: The string to analyze.

        Returns:
            A dictionary where keys are the duplicate letters and values are their counts.
            Returns an empty dictionary if there are no duplicates.
            Returns None if the input is not a string.
        """

        if not isinstance(self._text, str):
            return None

        letter_counts = {}
        duplicate_counts = {}

        input_string = self._text.lower()  # Case-insensitive counting

        for letter in input_string:
            if 'a' <= letter <= 'z':  # Only consider letters
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1

        # Identify duplicates and create a new dictionary
        for letter, count in letter_counts.items():
            if count > 1:
                duplicate_counts[letter] = count

        return duplicate_counts

class Winners:
    def __init__(self):
        self._sentences = []

    def add(self, sentence):
        self._sentences.append(sentence)
        # print(sentence.get_text())

    def get_missing_winners(self, _count):
        sorted_winners = sorted(self._sentences, key=lambda item: item.get_missing_count())
        return sorted_winners[:_count]
    
    def get_duplicate_winners(self, _count):
        sorted_winners = sorted(self._sentences, key=lambda item: item.get_duplength())
        return sorted_winners[:_count]

    def length(self):
        return len(self._sentences)

the_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Golden sunlight danced upon the rippling, turquoise ocean waves.",
    "Old photographs revealed stories of laughter, love, and bygone days.",
    "The majestic mountains stood silently guarding the peaceful valley below.",
    "She expertly crafted intricate jewelry using shimmering silver and gold.",
    "Dark storm clouds gathered quickly, threatening a powerful summer rain.",
    "Street food tasted wonderfully fresh.",
    "Remote islands felt truly peaceful.",
    "The axel, quickly! Jumping fences with brave, zealous strides."
]

winners = Winners()
for sentence in the_sentences:
    try:
        winners.add(Sentence(sentence))
    except Exception as e:
        print(e)


missing_winners = winners.get_missing_winners(5)
for winner in missing_winners:
    print(f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}'")

print("****************************")
duplicate_winners = winners.get_duplicate_winners(5)
for winner in duplicate_winners:
    print(f"duplicates: {winner.get_duplength()} {winner.get_text()} '{winner.get_dupkeys()}' missing {winner.get_missing_count()} '{winner.get_missing()}'")



missing 0 The quick brown fox jumps over the lazy dog. '' duplicates: 6 'theuro'
missing 0 The axel, quickly! Jumping fences with brave, zealous strides. '' duplicates: 11 'thealuicnsr'
missing 4 The majestic mountains stood silently guarding the peaceful valley below. 'kqxz' duplicates: 15 'themasicoundlyg'
missing 4 She expertly crafted intricate jewelry using shimmering silver and gold. 'bkqz' duplicates: 14 'shertlycadingm'
missing 5 Dark storm clouds gathered quickly, threatening a powerful summer rain. 'bjvxz' duplicates: 16 'darkstomclughein'
****************************
duplicates: 6 The quick brown fox jumps over the lazy dog. 'theuro' missing 0 ''
duplicates: 8 Street food tasted wonderfully fresh. 'strefodl' missing 12 'bcgijkmpqvxz'
duplicates: 8 Remote islands felt truly peaceful. 'retslafu' missing 10 'bghjkqvwxz'
duplicates: 11 The axel, quickly! Jumping fences with brave, zealous strides. 'thealuicnsr' missing 0 ''
duplicates: 14 Old photographs revealed stories of laug

In [4]:
import threading
import time
import random

timeout = 7*60*60
winning_sentences = Winners()
running = True
previous_missing = []

def worker(client_id, url, model):
    """
    Simulates a client processing a task.
    
    Args:
        client_id: Unique identifier for the client.
        url: The URL associated with the client's task.
        task_duration_range: A tuple representing the minimum and maximum duration of the task.
    """
    client = Client(host=url)
    i = 0
    while True:
        global running
        global winning_sentences
        global previous_missing
        i = i + 1
        if(running != True):
           print(f"returning client {client_id} url {url} model {model} i {i} running: {running}")
           return
        try:
            # print(f"client {client_id} url {url} model {model} i {i} running: {running}")
            sentence = Sentence(generate_response(client, get_prompt(), model).strip().replace("\n", " "))
            # print(sentence.get_text())
            winning_sentences.add(sentence)
            
            # print(f"There are {winning_sentences.length()} sentences in the list.")
            missing_winners = winning_sentences.get_missing_winners(4)
            if(missing_winners != previous_missing):
                previous_missing = missing_winners
                for winner in missing_winners:
                    print(f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}'")
                
                print("****************************\n")

            time.sleep(2)

        except Exception as e:
            dummy = 1 # Ignore all exceptions
            # print(f"client {client_id} url {url} {e}")

    return
        


def main():
    """
    Creates and starts multiple client threads.
    """

    clients = [
        {"id": 1, "url": "http://192.168.137.118:11434", "model": "gemma3:12b"},
        {"id": 2, "url": "http://192.168.137.119:11434", "model": "gemma3:4b"},
        {"id": 3, "url": "http://192.168.137.117:11434", "model": "gemma3:4b"},
    ]

    global running
    running = True
    threads = []
    for client in clients:
        thread = threading.Thread(target=worker, args=(client["id"], client["url"], client["model"]))
        threads.append(thread)
        thread.start()

    try:
        time.sleep(timeout)
        running = False

        # Wait for all threads to finish
        for thread in threads:
            thread.join()

        print("All threads have stopped.")
        missing_winners = winning_sentences.get_missing_winners(10)
        for winner in missing_winners:
            print(f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}'")

        print("****************************")
        duplicate_winners = winning_sentences.get_duplicate_winners(5)
        for winner in duplicate_winners:
            print(f"duplicates: {winner.get_duplength()} {winner.get_text()} '{winner.get_dupkeys()}' missing {winner.get_missing_count()} '{winner.get_missing()}'")
        print("============================\n\n")


    except KeyboardInterrupt:
        print("Stopping threads due to KeyboardInterrupt...")
        running = False
        for thread in threads:
            thread.join()
        print("All threads stopped after KeyboardInterrupt")

if __name__ == "__main__":
    main()

missing 7 Ultimately, achieve overall health through balanced and mindful sustenance. 'jkpqwxz' duplicates: 15 'ultimaechvornds'
****************************

missing 7 Ultimately, achieve overall health through balanced and mindful sustenance. 'jkpqwxz' duplicates: 15 'ultimaechvornds'
missing 11 To settle the internal conflict, let's finally, gently split ties. 'bdjkmquvwxz' duplicates: 11 'toselinacfy'
****************************

missing 7 Ultimately, achieve overall health through balanced and mindful sustenance. 'jkpqwxz' duplicates: 15 'ultimaechvornds'
missing 7 Legal sanction protects fragile life, ensuring freedom’s quiet, valuable space. 'hjkwxyz' duplicates: 14 'legasnctioprfu'
missing 11 To settle the internal conflict, let's finally, gently split ties. 'bdjkmquvwxz' duplicates: 11 'toselinacfy'
****************************

missing 7 Ultimately, achieve overall health through balanced and mindful sustenance. 'jkpqwxz' duplicates: 15 'ultimaechvornds'
missing 7 Legal sanc